# find_github_repo

## Goals

- To extract the names of github repositories from Pubmed abstracts that mention github.
- To use the github api to get author and contribution data about those repositories
- To create a dataset linking the two

The bit that reads in xml files and extracts article information to a data frame is from Kevin's xml_parsing script

First, set working directory

In [11]:
import os
os.chdir("../data/")
os.listdir("./")

['github_pubs.xml', 'README.md']

Next, import libraries needed for xml parsing

In [12]:
import xml.etree.ElementTree as ET
import datetime

Create Article class definition

In [13]:
class Article(object):
    """Container for publication info"""
    def __init__(self, pmid, pubdate, journal, title, abstract, authors):
        self.pmid = pmid
        self.pubdate = pubdate
        self.journal = journal
        self.title = title
        self.abstract = abstract
        self.authors = authors
    def __repr__(self):
        return "<Article PMID: {}>".format(self.pmid)

    def get_authors(self):
        for author in self.authors:
            yield author["Last"], author["First"]

Article generator function

In [14]:
def parse_pubmed_xml(xml_file):
    xml_handle = ET.parse(xml_file)
    root = xml_handle.getroot()

    for Citation in root.iter("MedlineCitation"):
        pmid = Citation[0].text
        pubdate = datetime.date(
            int(Citation[1][0].text),  # year
            int(Citation[1][1].text),  # month
            int(Citation[1][2].text)  # day
            )
        
        Journal = next(Citation.iter("Journal"))

        journal_title = Journal.find("ISOAbbreviation").text
        article_title = next(Citation.iter("ArticleTitle")).text
        
        abstract = next(Citation.iter("AbstractText")).text
        try:
            authors = [{
                "Last": Author.find("LastName").text,
                "First": Author.find("ForeName").text
                   } for Author in Citation.iter("Author")]
        except:
           continue
        
        yield Article(pmid, pubdate, journal_title, article_title, abstract, authors)

Make data frame (this differs a bit from Kevin's code by including abstract also)

In [15]:
import pandas as pd
df = pd.DataFrame()
col_names = ["Date", "Journal", "Authors","Abstract"]

for article in parse_pubmed_xml('github_pubs.xml'):
    row = pd.Series([article.pubdate, article.journal, [(author[0], author[1]) for author in article.get_authors()],
                     article.abstract],name=article.pmid, index=col_names)
    df = df.append(row)

df.head()

,Abstract,Authors,Date,Journal
26357045,Stability and sensitivity analyses of biologic...,"[(Shiraishi, Fumihide), (Yoshida, Erika), (Voi...",2015-09-11,IEEE/ACM Trans Comput Biol Bioinform
25601296,Most electronic data capture (EDC) and electro...,"[(Dixit, Abhishek), (Dobson, Richard J B)]",2015-01-20,JMIR Med Inform
25558360,Remotely sensed data - available at medium to ...,"[(Tuck, Sean L), (Phillips, Helen Rp), (Hintze...",2015-01-05,Ecol Evol
25553811,Whole-genome bisulfite sequencing (WGBS) is an...,"[(Chen, Junfang), (Lutsik, Pavlo), (Akulenko, ...",2015-01-02,J Bioinform Comput Biol
25549775,A number of computational approaches have been...,"[(Manini, Simone), (Antiga, Luca), (Botti, Lor...",2015-06-09,Ann Biomed Eng


In [ ]:
Check the content of the first abstract

In [26]:
df.iat[0,0]

'Stability and sensitivity analyses of biological systems require the ad hocwriting of computer code, which is highly dependent on the particular model and burdensome for large systems. We propose a very accurate strategy to overcome this challenge. Its core concept is the conversion of the model into the format of biochemical systems theory (BST), which greatly facilitates the computation of sensitivities. First, the steady state of interest is determined by integrating the model equations toward the steady state and then using a Newton-Raphson method to fine-tune the result. The second step of conversion into the BST format requires several instances of numerical differentiation. The accuracy of this task is ensured by the use of a complex-variable Taylor scheme for all differentiation steps. The proposed strategy is implemented in a new software program, COSMOS, which automates the stability and sensitivity analysis of essentially arbitrary ODE models in a quick, yet highly accurate

Find address of github repository from abstract by searching for the "https://github.com" and then for the next space (this seems to work even if the abstract ends with the github address, so there is no space at the end).

TO DO: make this into a for loop!

In [53]:
len(df.index)

402

In [51]:
abstract = df.iat[0,0]
start_index = abstract.find("https://github.com")
url_to_end = abstract[start_index:]
space_index = url_to_end.find(" ")
github_url = url_to_end[:space_index]


22
https://github.com/foo
